In [ ]:
import os

os.chdir("/data/FUTURES_case_study/")

In [ ]:
import subprocess
import sys
import pathlib
import json
import pandas as pd
from IPython.display import Image

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass8", "--config", "python_path"], text=True).strip()
)

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
session = gj.init(".", "FUTURES", "figure")

In [ ]:
!g.region region=nashville -p

In [ ]:
%%writefile frame.txt
color 255:255:255
width 4
  polyline
   807097 1497916
   807097 1519224
   833094 1519224
   833094 1497916
    807097 1497916

In [ ]:
label|symbol_name|size|color_type|feature_color|fill_color|line_width|geometry_type|feature_count

In [ ]:
%%writefile legend.txt
2019|legend/area|100|lf|none|#9c9c9c|1|area|1
2050|legend/area|25|lf|none|#F6ABFF|1|area|1

In [ ]:
%%writefile legend.txt
Simulated development in 2050    |legend/area|25|lf|none|#F6ABFF|1|area|1

In [ ]:
gs.run_command("g.region", n=1528356, s=1450112, w=770446, e=865910, align="probability@tutorial", save="nashville")
gs.write_command("r.colors", map="urban_2019@tutorial", rules="-",stdin="0 white\n1 #9c9c9c")
gs.run_command("r.colors", map="probability@tutorial", color="magma")
m = gj.Map(saved_region="nashville", width=1000)
m.d_background(color="#2D2D2D")
m.d_rast(map="probability@tutorial")
m.d_rast(map="urban_2019@tutorial", values=1)
m.d_vect(map="tl_2021_47_prisecroads", color="200:200:200", width=1)

m.d_graph(input="frame.txt", flags="m")
# m.d_legend(raster="probability@tutorial", color="black", at="4,8,1,38", title="Development probability", #at="1,15,1,4"
#            flags="b", border_color="230:230:230", bgcolor="230:230:230", fontsize=18, labelnum=2, digits=0, title_fontsize=25, font="Lato-Regular")
# m.d_barscale(style="solid", at=[40,3], font="Lato-Regular", length=20, unit="kilometers", fontsize=17, bgcolor="230:230:230")
m.d_legend(raster="probability@tutorial", color="black", at="10,14,2,25.7", title="Development probability", #at="1,15,1,4"
           flags="b", border_color="230:230:230", bgcolor="230:230:230", fontsize=17, labelnum=2, digits=0, title_fontsize=17, font="Lato-Regular")
m.d_barscale(style="solid", at=[0,6], font="Lato-Regular", length=25, unit="kilometers", fontsize=17, bgcolor="230:230:230", text_position="under")
#m.d_text(text="A", at="94.1,6", align="cc", color="black", font="Lato-Bold", size=10)
m.d_text(text="A", at="94,6", align="cc", color="white", font="Lato-Bold", size=8)
m.d_text(flags="g", text="B", at="830094,1502016", align="cc", color="white", font="Lato-Bold", size=8)

m.save("nashville.png")
m.show()


In [ ]:

gs.run_command("g.region", n=1519224, s=1497916, w=807097, e=833094, align="probability@tutorial", save="nashville_zoom")
gs.write_command("r.colors", map="out_seed_1@results_1", rules="-", stdin="-1 #171717\n0 #878686\n1 #F6ABFF\n 100 #F6ABFF")
m = gj.Map(saved_region="nashville_zoom", width=1000)
m.d_background(color="#2D2D2D")
m.d_rast(map="out_seed_1@results_1")
#m.d_rast(map="urban_2019@tutorial", values=1)
m.d_vect(map="tl_2021_47_prisecroads", color="200:200:200", width=1)
#m.d_legend(raster="probability@tutorial", color="black", at="1,15,1,4", title="Probability", flags="b", border_color="white", fontsize=15, labelnum=2, digits=0)
m.d_legend_vect(flags="b", at=[1,13], symbol_size=36, fontsize=17, input="legend.txt",
               border_color="none", font="Lato-Regular", columns=1, bgcolor="230:230:230",)
# m.d_barscale(style="solid", at=[0,6], font="Lato-Regular", length=5, unit="kilometers", fontsize=17, bgcolor="230:230:230",)
m.d_barscale(style="solid", at=[0,6], font="Lato-Regular", length=10, unit="kilometers", fontsize=17, bgcolor="230:230:230", text_position="under")
m.d_text( text="B", at="94,6", align="cc", color="white", font="Lato-Bold", size=8)
m.save("nashville_zoomin.png")
m.show()

In [ ]:
#gs.run_command("r.colors", map="forest_loss", color="magma")
#gs.run_command("r.mask", raster="SE_states")
#gs.run_command("g.region", save="CONUS", vector="states", align="forest_loss@tutorial")
m = gj.Map(width=1000, saved_region="CONUS")
m.d_vect(map="states", fill_color="#2D2D2D", width=1, color="none")
#m.d_vect(map="states", fill_color="#171717", width=1, color="none")
#m.d_background(color="white")
m.d_rast(map="forest_loss")
#m.d_rast(map="urban_2019@tutorial", values=1)
m.d_vect(map="states", color="200:200:200", width=1, fill_color="none")
m.d_legend(raster="forest_loss", color="black", at="13,17,5,25", title="Forest loss in 2050 [km sq]", #at="1,15,1,4"
           flags="bt", border_color="white", fontsize=17, labelnum=5, digits=0, title_fontsize=17)
m.d_barscale(style="solid", at=[3.5,6], font="Lato-Regular",  fontsize=17, text_position="under")
m.save("forest_loss.png")
m.show()

In [ ]:
gs.run_command("g.region", flags="d", n="n+90000", w="w-90000", s="s-50000", e="e+50000", align="nlcd_2019", save="overview")
gs.write_command("v.in.ascii", flags="t", input="-", output="nashville", stdin="819538.3156168542,1492908.1911508846", sep="comma")
m = gj.Map(width=282, saved_region="overview")
#m.d_background(color="#2D2D2D")
#m.d_background(color="20:20:20")
m.d_background(color="230:230:230")
m.d_vect(map="states", color="200:200:200", width=1, fill_color="#9C9C9C")
#m.d_vect(map="SE_states", color="200:200:200", width=3, fill_color="#9C9C9C")
#m.d_vect(map="nashville", icon="basic/circle", size=15, color="#EA6F6D", fill_color="#EA6F6D")
m.d_vect(map="nashville", icon="basic/circle", size=15, color="white", fill_color="white")
m.d_text(flags="g", text="Nashville, TN", at="900038.3156168542,1492908.1911508846", align="ll", color="white", font="Lato-Bold")
m.save("overview.png")
m.show()

In [ ]:
from PIL import Image as PILImage

im1 = PILImage.open('nashville.png')
im2 = PILImage.open('overview.png')
im3 = PILImage.new('RGB', (im2.size[0] + 5, im2.size[1]+ 5), (200, 200, 200))
# im1.paste(im3)
im1.paste(im2, (0, 00))
im1.save("nashville_overview.png")
Image("nashville_overview.png")